In [1]:
import openai
import sys,os,os.path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

data = pd.read_csv("FilteredDataJuly19.csv") #reads in data
sector_keys = pd.read_csv("gicsectorkey_list.csv") #reads in keys
data = data.merge(sector_keys, how='left', on='gsector')
fraud_cases = data[data['misstate'] == 1]

In [2]:
fraud_cases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258 entries, 332 to 20427
Data columns (total 65 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   fyear                   258 non-null    int64  
 1   gvkey_x                 258 non-null    int64  
 2   misstate                258 non-null    int64  
 3   act                     258 non-null    float64
 4   ap                      258 non-null    float64
 5   at                      258 non-null    float64
 6   ceq                     258 non-null    float64
 7   che                     258 non-null    float64
 8   cogs                    258 non-null    float64
 9   csho                    258 non-null    float64
 10  dlc                     258 non-null    float64
 11  dltis                   258 non-null    float64
 12  dltt                    258 non-null    float64
 13  dp                      258 non-null    float64
 14  ib                      258 non-null    flo

In [3]:
fraud_cases = fraud_cases[['fyear','conm','sector_name']].drop_duplicates(subset=['conm'])

In [1]:
def fine_or_settlement_amount(year,company,sector_name,data):
    original_response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", "content": """{1} is a company in the {2} sector that was found in {0} to have committed fraud. 
                                    What was the settlement amount, 
                                    if no settlement amount them what's the fine?""".format(year,company,sector_name)}
      ]
    )
    text = original_response.choices[0].message.content
    amount = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {"role": "user", 
         "content": """
         {0} is the output from a ChatGPT response. I want the amount only. 
         For example if the total fine was $190 million USD. 
         I want the output as '190000000'. 
         If there is no amount, then respond with 0.""".format(text)}
      ]
    )
    amount_formatted = amount.choices[0].message.content
    print(amount_formatted)
    new_row = {}
    new_row.update([("conm",company),("amount",amount_formatted)])
    data.append(new_row)
    return data

data = []
fraud_cases.apply(lambda row: fine_or_settlement_amount(row.iloc[0],row.iloc[1],row.iloc[2],data),axis=1,result_type=None)
fine_settlement_amount = pd.DataFrame.from_dict(data)



In [15]:
data = []
fraud_cases.apply(lambda row: fine_or_settlement_amount(row.iloc[0],row.iloc[1],row.iloc[2],data),axis=1,result_type=None)
fine_settlement_amount = pd.DataFrame.from_dict(data)

1413000000
1200000
0
7000000
457000000
850000
0
0
0
2500000
0
3000000
120000000
0
0
12000000
10000000
0
0
21000000
126700000
2942000000
9700000
0
7300000
135000000
0
1700000000
8000000
0
20700000
100000000
44300000
0
14000000
16000000
203500000
0
0
15000000
1101000000
300000000
6600000
0
0
250000000
25000000
0
0
0
100000000
0
127000000
1000000
I'm sorry, but I cannot provide current, specific details about the settlement amount or fines related to Northwestern Energy Group Inc's fraud case from 2001.
1000000
225000000
62000000
0
0
3000000
0
0
0
0
0
500000
3000000
30000000
1210000000
3250000
0
The amount is 100000000.
0
7000000
0
20900000
75000001
0
1000000
253000000
82300000
11000000
965000000
5000000
0
853200000
42000000
0
0


In [26]:
fraud_cases = fraud_cases.merge(fine_settlement_amount, how='left', on='conm')
display(fine_settlement_amount)

,conm,amount
0,TENET HEALTHCARE CORP,1413000000
1,CHAMBERS DEVELOPMENT -CL A,1200000
2,GRACE (W R) & CO,0
3,PERRY DRUG STORES,7000000
4,WASTE MANAGEMENT INC-OLD,457000000
...,...,...
85,DXC TECHNOLOGY CO,0
86,PETROLEO BRASILEIRO SA- PETR,853200000
87,ORTHOFIX MEDICAL INC,42000000
88,TECH DATA CORP,0


In [24]:
display(fraud_cases)

,fyear,conm,sector_name,amount_x,amount_y
0,1990,TENET HEALTHCARE CORP,Health Care,1413000000,1413000000
1,1990,CHAMBERS DEVELOPMENT -CL A,Industrials,1200000,1200000
2,1991,GRACE (W R) & CO,Materials,0,0
3,1992,PERRY DRUG STORES,Consumer Staples,7000000,7000000
4,1992,WASTE MANAGEMENT INC-OLD,Industrials,457000000,457000000
...,...,...,...,...,...
85,2009,DXC TECHNOLOGY CO,Information Technology,0,0
86,2009,PETROLEO BRASILEIRO SA- PETR,Energy,853200000,853200000
87,2011,ORTHOFIX MEDICAL INC,Health Care,42000000,42000000
88,2012,TECH DATA CORP,Information Technology,0,0
